<a href="https://colab.research.google.com/github/MayyarAA/NaturalLangaugeProcessingBERT/blob/main/MSCI598ProjectBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [ ]:
!pip install transformers

In [2]:
import transformers
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd
import matplotlib.pyplot as plt
import sklearn    
from sklearn.model_selection import train_test_split

import transformers
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [ ]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [ ]:
classifier("Great! I'm so happy this exists.")

[{'label': 'POSITIVE', 'score': 0.9998798370361328}]

# Taking in the csv files

In [3]:
from google.colab import drive
drive.mount('/content/drive')     


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train_body = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_bodies.csv")

In [5]:
train_body = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_bodies.csv")
train_stance = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_stances.csv")

In [ ]:
train_body.head(5)

In [ ]:
train_stance.head(5)

In [ ]:
train_stance.loc[train_stance['Body ID']==712].head(2)

# Merge the stance & body on Body ID

In [9]:
full_sb = pd.merge(train_stance,train_body,on="Body ID")
full_sb.head(1)

,Headline,Body ID,Stance,articleBody
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\n\n...


In [ ]:
import re
def removeSpecailCharFromString(orgWord):
    if(len(orgWord)>100):
      orgWord = orgWord[:100];
    orgWord = orgWord.rstrip('\n')
    return re.sub(r"[^a-zA-Z0-9]+", ' ', orgWord).strip()

In [ ]:
inputs = tokenizer(full_sb.iloc[0].Headline, return_tensors="pt")
inputs

{'input_ids': tensor([[  101,  2610,  2424,  3742,  9729,  2007,  2012,  2560,  1005,  2321,
          4230,  1005,  2379,  3290,  2237,  2073,  4724,  2493,  5419,  2044,
          2610, 13249,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits

# Set up for Simple BERT

In [ ]:
### using simple BERT
!pip install simpletransformers
from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)

In [ ]:
# Preparing train data
train_data = [
    [
        "Aragorn was the heir of Isildur",
        "Gimli fought with a battle axe",
        1,
    ],
    [
        "Frodo was the heir of Isildur",
        "Legolas was an expert archer",
        0,
    ],
]
train_df = pd.DataFrame(train_data)
train_df.columns = ["text_a", "text_b", "labels"]

# Preparing eval data
eval_data = [
    [
        "Theoden was the king of Rohan",
        "Gimli's preferred weapon was a battle axe",
        1,
    ],
    [
        "Merry was the king of Rohan",
        "Legolas was taller than Gimli",
        0,
    ],
]
eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["text_a", "text_b", "labels"]

# Optional model configuration
#model_args = ClassificationArgs(num_train_epochs=1)


# Setup pearson_corr & spearman_corr for Simple BERT

In [7]:
from scipy.stats import pearsonr, spearmanr

def pearson_corr(preds, labels): return pearsonr(preds, labels)[0]

def spearman_corr(preds, labels): return spearmanr(preds, labels)[0]

# Trim the size of the articleBody 

In [10]:
print(len(full_sb.iloc[0]['articleBody']))

1093


In [11]:
#make full_sb shorter body

full_sb['articleBody'] = full_sb['articleBody'].str.slice(0,50)
full_sb.head(1)

,Headline,Body ID,Stance,articleBody
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\n\n...


In [12]:
print(len(full_sb.iloc[0]['articleBody']))

50


In [13]:
# train_df_simple = full_sb[['Headline', 'articleBody','Stance']].copy()
# train_df_simple.columns = ["text_a", "text_b", "labels"]
# train_df_simple.head(1)
smaller_sb = full_sb.iloc[0:1000]
print(len(smaller_sb))

1000


In [14]:
train_df_simple = smaller_sb[['Headline', 'articleBody','Stance']].copy()

In [ ]:
#print(train_df_simple.iloc[0])

In [15]:
#turn stance into integer value 
train_df_simple['Stance'] = train_df_simple['Stance'].replace({'unrelated':0,'agree': 1, 'disagree':2 , 'discuss':3})
print(train_df_simple.iloc[0])

Headline       Police find mass graves with at least '15 bodi...
articleBody    Danny Boyle is directing the untitled film\n\n...
Stance                                                         0
Name: 0, dtype: object


# Create BERT Simple Model


In [18]:
# Create a ClassificationModel
model_args = {
    'fps16':True,
    'num_train_epochs': 5,
    'learning_rate': 1e-5,
    'overwrite_output_dir': True,
    'reprocess_input_data':True,
    'process_count':10,
    'train_batch_size':6,
    'eval_batch_size':6,
    'max_seq_length':50
}

model = ClassificationModel("roberta", "roberta-base",use_cuda=True,num_labels=4,args=model_args)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [20]:
train_df_simple.columns = ['text_a',"text_b","labels"]

In [21]:
#model.train_model(train_df=train_df_simple, eval_df=eval_df, pearson_corr=pearson_corr, spearman_corr=spearman_corr)
model.train_model(train_df_simple)

  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/167 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/167 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/167 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/167 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/167 [00:00<?, ?it/s]

(835, 0.34783089328403893)

# Test Simple BERT Model


## Prepare Testing Dataframe


In [39]:
def generate_df_from_csv(path):
  df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_bodies.csv")

def produce_set_for_simpleBERT(stance,body):
  full_sb = pd.merge(train_stance,train_body,on="Body ID")
  #make full_sb shorter body
  full_sb['articleBody'] = full_sb['articleBody'].str.slice(0,50)
  df_simple= full_sb[['Headline', 'articleBody','Stance']].copy()
  df_simple['Stance'] = df_simple['Stance'].replace({'unrelated':0,'agree': 1, 'disagree':2 , 'discuss':3})
  df_simple.columns = ['text_a',"text_b","labels"]
  return df_simple

def df_to_listOflist(df):
  df_2d = df[['text_a','text_b']].copy()
  listOfList = df_2d.to_numpy().tolist()
  return listOfList;

In [34]:
stance_comp = generate_df_from_csv("/content/drive/MyDrive/Colab Notebooks/competition_bodies.csv")
body_comp = generate_df_from_csv("/content/drive/MyDrive/Colab Notebooks/competition_bodies.csv")
df_comp = produce_set_for_simpleBERT(stance_comp,body_comp)
df_comp.head(1)

,text_a,text_b,labels
0,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled film\n\n...,0


In [146]:
print(len(df_comp))

49972


In [40]:
comp_list = df_to_listOflist(df_comp)
#print(comp_list.pop())

['If you want an Apple Watch, you’ll probably be trekking to an Apple Store', 'We still don’t know the exact date the Apple Watch']


## Testing BERT Simple Model


In [41]:
# test the model
predictions, raw_outputs = model.predict(comp_list)

  0%|          | 0/49971 [00:00<?, ?it/s]

  0%|          | 0/8329 [00:00<?, ?it/s]

In [55]:
print(f'predictions {predictions.flat[1]} + predictions length {len(predictions)} of type {type(predictions)}')

predictions 3 + predictions length 49971 of type <class 'numpy.ndarray'>


## Determine Testing Accuracy of BERT Simple Model


In [157]:
import numpy as np
def turn_list_to_df(listObj):
  #df = pd.DataFrame([["a","a"]],columns=['A','B'])
  df = pd.DataFrame()
  # for listRow in listObj:
  #   df.loc[len(df)] = listRow
  #while count < len(listObj):
  count=0
  while count < 10:
    dfFromList = pd.DataFrame()
    dfFromList['a'] = listObj[count][0]
    dfFromList['b'] = listObj[count][1]
    #print(listObj[count])
    #print(dfFromList)
    #df.loc[len(df)] = listObj[count]
    df= df.append(dfFromList)    
    count= count+1
  return df;
def determine_acc_simple_bert(listObj,pred):
  df = turn_list_to_df(listObj)
  df = pd.DataFrame(pred)
  pred_df = pd.DataFrame(pred)
  #df['prediction'] = pred.tolist()
  df = df.join(pred_df)
  print(df.head(0))
  # df['prediction'] = df['prediction'].replace({0:'unrelated',1:'agree', 2:'disagree' , 3:'discuss'})
  # df['correct'] = np.where(df['stance']==df['pred'],1,0)
  # acc = (df['correct'].sum()/(len(df)))
  # return acc
  return 0

def determine_acc_simple_bert(listObj,pred):
  dfStart = pd.DataFrame(listObj, columns=list('xy'))  
  pred_df = pd.DataFrame(pred)
  #df['prediction'] = pred.tolist()
  #dfStart = df.join(pred_df)
  predMini = pred[:49964]
  dfStart['pred']= predMini
  print(dfStart.columns)
  #print(dfStart.head(0))
  # df['prediction'] = df['prediction'].replace({0:'unrelated',1:'agree', 2:'disagree' , 3:'discuss'})
   #dfStart['correct'] = np.where(dfStart['stance']==dfStart['pred'],1,0)
   #acc = (df['correct'].sum()/(len(df)))
  # return acc
  return 0

def determine_acc_simple_bert_v2(df,pred):
  #predMini = pred[:49971]
  dfMini = df[:49971]
  #df['pred']= predMini
  dfMini['pred'] = pred
  dfMini['correct'] = np.where(dfMini['labels']==dfMini['pred'],1,0)
  acc = (dfMini['correct'].sum()/(len(dfMini)))
  return acc

In [79]:
print(type(comp_list.pop()[0]))

<class 'str'>


In [114]:
print(comp_list[0])

['Seth Rogen to Play Apple’s Steve Wozniak', 'Danny Boyle is directing the untitled film\n\nSeth R']


In [124]:
dfss = turn_list_to_df(comp_list)
print(dfss.head(3))

Empty DataFrame
Columns: [a, b]
Index: []


In [70]:
print((comp_list[0]))

['Seth Rogen to Play Apple’s Steve Wozniak', 'Danny Boyle is directing the untitled film\n\nSeth R']


In [71]:
df = pd.DataFrame(comp_list[0])
#df = df.transpose()
print(df.head(0))

Empty DataFrame
Columns: [0]
Index: []


In [129]:
print(dfNow.size)

99928


In [130]:
print(len(dfNow.columns))

2


In [134]:
print(dfNow.iloc[0,:])

x             Seth Rogen to Play Apple’s Steve Wozniak
y    Danny Boyle is directing the untitled film\n\n...
Name: 0, dtype: object


In [ ]:
dfNow = pd.DataFrame(comp_list, columns=list('xy'))
#print(dfNow)

In [139]:
print(f'len of comp_list is {len(comp_list)} len of predictions {len(predictions)}')

len of comp_list is 49964 len of predictions 49971


In [145]:
acc =determine_acc_simple_bert(comp_list,predictions)
print(f'acc {acc}')

Index(['x', 'y', 'pred'], dtype='object')
acc 0


In [153]:
print(df_comp.head(2))

                                              text_a  \
0  Police find mass graves with at least '15 bodi...   
1           Seth Rogen to Play Apple’s Steve Wozniak   

                                              text_b  labels  
0  Danny Boyle is directing the untitled film\n\n...       0  
1  Danny Boyle is directing the untitled film\n\n...       3  


In [154]:
print(predictions[0])

0


In [159]:
acc2 = determine_acc_simple_bert_v2(df_comp,predictions)
print(f'acc {acc2}')

acc 0.8074683316323468


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:
# Preparing eval data
eval_data = [
    [
        "Theoden was the king of Rohan",
        "Gimli's preferred weapon was a battle axe",
        1,
    ],
    [
        "Merry was the king of Rohan",
        "Legolas was taller than Gimli",
        0,
    ],
]
eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["text_a", "text_b", "labels"]

In [24]:
print(f'result {result}')
print(f'model_outputs {model_outputs}')

result {'mcc': 0.0, 'eval_loss': 3.824929714202881}
model_outputs [[ 5.89453125 -1.75097656 -1.97949219 -2.04882812]
 [ 5.76953125 -1.7265625  -2.00390625 -1.94335938]]


In [25]:
# Make predictions with the model
predictions, raw_outputs = model.predict(
    [
        [
            "Legolas was an expert archer",
            "Legolas was taller than Gimli",
        ],[
            "Legolas was an expert archer",
            "Legolas was an expert archer",
        ],[
           "Theoden was the king of Rohan",
           "Gimli's preferred weapon was a battle axe"
           ]
    ]
)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
print(f'predictions {predictions}')

predictions [3 3 0]


In [ ]:
model.train_model(train_df, eval_df=eval_df, pearson_corr=pearson_corr, spearman_corr=spearman_corr)

In [ ]:
#data cleaning


#tokenize use BERT

#load model
#fine tune model 